In [ ]:
import os
import faiss
from llama_index.core import (
    load_index_from_storage,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.llms.openai import OpenAI

EMBEDDING_DIMENSIONS = {
    "sentence-transformers/all-MiniLM-L6-v2": 384,
    "sentence-transformers/all-mpnet-base-v2": 768,
    "text-embedding-3-small": 1536,
}

In [7]:
input_dir = "/Users/goeksu/dev/interactive_rag/data/landing"
persist_root = "/Users/goeksu/dev/interactive_rag/data/index"
chunk_size = 256
model_name = "text-embedding-3-small"

# File ingestion
# Simple directory reader seems to be sufficient for this task.
reader = SimpleDirectoryReader(input_dir=input_dir)
documents = reader.load_data()
# Split docs to nodes and create vector index
# Just setting a sliding window chunk here, nothing fancy.
node_parser = SimpleNodeParser.from_defaults(
    chunk_size=chunk_size, chunk_overlap=int(chunk_size * 0.2)
)
nodes = node_parser.get_nodes_from_documents(documents=documents)

In [8]:
# Initialize embedding model
if model_name.startswith("text-embedding-3"):
    embedding_model = OpenAIEmbedding(model=model_name)
else:
    embedding_model = HuggingFaceEmbedding(model_name=model_name)

In [9]:
# Initialize a faiss vector store
d = EMBEDDING_DIMENSIONS[model_name]
faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(
    nodes=nodes, storage_context=storage_context, embed_model=embedding_model
)
# Save index to disk
persist_dir = os.path.join(persist_root, model_name)
index.storage_context.persist(persist_dir=persist_dir)

In [11]:
# interact with index
chat_engine = index.as_chat_engine(llm=OpenAI(model="gpt-4o-mini"))

In [ ]:
response = chat_engine.chat("What sort of information about ChemNovus can you provide?")
print(response)

ChemNovus Incorporated is dedicated to compliance with all relevant federal, state, and local laws, particularly concerning background checks to ensure a fair and non-discriminatory hiring process. Here are some key aspects of the company:

- **Compensation Philosophy**: ChemNovus aims to provide competitive salaries that align with industry standards and individual performance. They conduct regular market surveys to maintain competitiveness.

- **Salary Reviews**: The company conducts annual salary reviews, offering merit-based increases based on performance evaluations that consider factors such as job knowledge and teamwork. They also have variable compensation programs to reward exceptional contributions.

- **Employee Health Benefits**: ChemNovus offers a comprehensive health insurance plan for full-time employees and their eligible dependents.

- **Performance Management**: The company maintains a transparent and objective performance management process, encouraging employee part

In [14]:
response = chat_engine.chat("Can you give me more information about health benefits?")
print(response)

ChemNovus Incorporated provides a comprehensive health benefits package that includes three medical plan options through a partnership with TheInsuranceCompany Ltd. Here are the details:

1. **Plan A**: 
   - A traditional PPO plan.
   - Features a broad network of providers.
   - Comes with moderate deductibles and copays.
   - Covers preventive care services, doctor visits, hospital stays, and prescription drugs.

2. **Plan B**: 
   - A high-deductible health plan (HDHP).
   - Includes a health savings account (HSA) option, allowing employees to contribute pre-tax dollars for eligible medical expenses.
   - Also covers preventive care services, doctor visits, hospital stays, and prescription drugs.

The specific differences in deductibles, copays, and coinsurance rates vary between the plans, allowing employees to choose the option that best fits their healthcare needs and financial situation.


In [ ]:
response = chat_engine.chat(
    "Are these two the only health benefits available, or is there more?"
)
print(response)

In addition to the two medical plan options (Plan A and Plan B), ChemNovus Incorporated offers supplementary health insurance options that enhance coverage. These additional health benefits may include:

- **Dental Care**: Coverage for dental services and treatments.
- **Vision Care**: Coverage for eye exams, glasses, and contact lenses.
- **Alternative Therapies**: Options for therapies that may not be covered under standard medical plans.
- **Private Hospital Rooms**: Coverage for private accommodations during hospital stays.

These supplementary benefits provide employees with more comprehensive health coverage beyond the basic medical plans.


In [19]:
# Load index and interact
vector_store = FaissVectorStore.from_persist_dir(persist_dir)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir=persist_dir
)
index_load = load_index_from_storage(storage_context=storage_context)

# interact with index
chat_engine_2 = index_load.as_chat_engine(
    llm=OpenAI(model="gpt-4o-mini"), chat_mode="condense_plus_context"
)

In [20]:
response = chat_engine_2.chat(
    "What sort of information about ChemNovus can you provide?"
)
print(response)

I can provide information about ChemNovus Incorporated's HR policies and procedures as outlined in their handbook. For instance, the handbook discusses the performance management process, which includes performance ratings and calibration, development planning, and how these evaluations can impact compensation and career progression. 

Employees receive performance ratings that reflect their contributions during a specific period, and these ratings are calibrated across teams to ensure fairness. Additionally, the performance evaluation process helps identify areas for professional development and creates plans to address skill gaps. 

Moreover, the results of these evaluations can influence decisions regarding merit-based salary increases, bonuses, promotions, and other career advancement opportunities. 

The handbook also emphasizes the importance of open communication during disciplinary actions, allowing employees the right to appeal decisions through established grievance procedure